In [1]:
from pathlib import Path
from dotenv import dotenv_values, load_dotenv

import torch
import coref_ds
from coref_ds.text import Text
from coref_ds.mmax.mmax_doc import MmaxDoc
from coref_ds.tei.tei_doc import TEIDocument

from plotkarzyna.model import get_model, predict

try:
    local_config = dotenv_values(".env")
except FileNotFoundError:
    local_config = dict()
load_dotenv()

[nltk_data] Downloading package punkt to /home/ksaputa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
n_samples = 100_000_000
test_paths = list(
    (Path(local_config['PCC_ROOT']) / 'test').iterdir()
    )[:n_samples]

In [3]:
tei_docs = []
for p in test_paths:
    try: 
        m = TEIDocument(p)
        tei_docs.append(m)
    except Exception as e:
        print(e)

In [4]:
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large-2/checkpoint-6240'
tokenizer, model = get_model(checkpoint_path)

In [5]:
preds = []
for ind, doc in enumerate(tei_docs):
    print(ind, end=' ')
    pred = predict(doc.text.segments, model, tokenizer)
    preds.append(pred)

0 (392, 392)
['</s>']
1 (378, 378)
['</s>']
2 3 4 (368, 368)
['</s>']
5 (353, 353)
['</s>']
6 7 (450, 451)
['Pawel', '</s>']
8 9 10 (335, 335)
['</s>']
11 12 (338, 338)
['</s>']
13 (284, 284)
['</s>']
14 

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


(291, 292)
['mnie', '</s>']
15 (489, 489)
['</s>']
(491, 494)
['<pad>', '<pad>', '<pad>', '<pad>']
(496, 497)
['<pad>', '<pad>']
(504, 504)
['<pad>']
(489, 489)
['</s>']
(491, 494)
['<pad>', '<pad>', '<pad>', '<pad>']
(496, 497)
['<pad>', '<pad>']
(504, 504)
['<pad>']
(988, 996)
['</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
(998, 998)
['<pad>']
(1000, 1001)
['<pad>', '<pad>']
(1001, 1001)
['<pad>']
(1004, 1007)
['<pad>', '<pad>', '<pad>', '<pad>']
(489, 489)
['</s>']
(491, 494)
['<pad>', '<pad>', '<pad>', '<pad>']
(496, 497)
['<pad>', '<pad>']
(504, 504)
['<pad>']
(988, 996)
['</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
(998, 998)
['<pad>']
(1000, 1001)
['<pad>', '<pad>']
(1001, 1001)
['<pad>']
(1004, 1007)
['<pad>', '<pad>', '<pad>', '<pad>']
(1514, 1514)
['</s>']
(489, 489)
['</s>']
(491, 494)
['<pad>', '<pad>', '<pad>', '<pad>']
(496, 497)
['<pad>', '<pad>']
(504, 504)
['<pad>']
(988, 996)
['</s>', '<pad>', '<pad>

In [11]:
for mention in preds[-1][-1]:
    print(' '.join(mention))

Katowicach
zespołów mieszkaniowych
15 placów budowy zespołów mieszkaniowych
budowy zespołów mieszkaniowych
katowickim Osiedlu Ptasim
Firma Holdimex , budująca na katowickim Osiedlu Ptasim
nowe ulice
13-hektarowy teren , na którym powstają nowe ulice
zrealizowała
400 mieszkań : przy ul Zimorodków , Gawronów , Kormoranów , Drozdów
ul Zimorodków , Gawronów , Kormoranów , Drozdów
Zimorodków , Gawronów , Kormoranów , Drozdów
przygotowaniu
ulica Pliszek
Pliszek
Drugie tyle
Holdimex
Osiedlu Ptasim
Osiedlu Ptasim
urozmaiconej architekturze
trzy- lub pięcio piętrowych budynkach o urozmaiconej architekturze
pięcio piętrowych budynkach o urozmaiconej architekturze
urozmaiconej architekturze
tu
cierpliwość
szybkim tempie
mieszkania , które są jeszcze na papierze , a będą gotowe dopiero w 2008 r .
papierze , a będą gotowe dopiero w 2008 r . Ich cena
dotychczasowej
tak zwanym stanie deweloperskim
metr kwadratowy w tak zwanym stanie deweloperskim
białego montażu
drzwi wewnętrznych
pokryć podłogowych


In [7]:
pred = preds[-1]
tokens, offset = pred[0], pred[-1]

In [8]:
offset[0]

['Katowicach']

In [9]:
for t_offset in offset[0]:
    print(tokens[t_offset[0]:(t_offset[1]+1)], t_offset)

TypeError: can only concatenate str (not "int") to str